In [1]:
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime,timedelta
import time
import talib as ta

In [2]:
#tz = pytz.timezone('America/Sao_Paulo')
ativo = 'WIN@N'
#ativos = ['EURUSD']



if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

In [3]:

def comprar(symbol,stop,hora_fechamento):
    point = mt5.symbol_info(symbol).point
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 1.0,
        "type": mt5.ORDER_TYPE_BUY,
        "price": mt5.symbol_info_tick(symbol).ask,
        "sl": stop,
        "tp": 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "python script open",
        "type_time": mt5.ORDER_TIME_SPECIFIED,
        "expiration": hora_fechamento,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result = mt5.order_send(request)
    print(result)

def vender(symbol,stop,hora_fechamento):
    point = mt5.symbol_info(symbol).point
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 1.0,
        "type": mt5.ORDER_TYPE_SELL,
        "price": mt5.symbol_info_tick(symbol).bid,
        "sl": stop,
        "tp": 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "python script open",
        "type_time": mt5.ORDER_TIME_SPECIFIED,
        "expiration": hora_fechamento,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result = mt5.order_send(request)
    print(result)

In [ ]:
def verificar(ativo, hora_fechamento):

    df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 200))
    df['MA_9'] = ta.EMA(df['Close'], timeperiod = 9)
    df['MA_9_diff_value'] = df['MA_9'].diff(periods = 1)
    df['MA_9_tend'] = df['MA_9_diff_value'].apply(lambda x: 1 if x > 0 else -1)
    df['MA_9_tend_diff'] = df['MA_9_tend'].diff()
    df['MA_100'] = ta.SMA(df['Close'],timeperiod=100)
    df['tend'] = df.apply(lambda x: 1 if x['Close'] > x['MA_100'] else 0, axis = 1)

    df['acao'] = df.apply(lambda x: 'sell' if (x['MA_9_tend_diff'] == 2 and x['tend'] == 1) 
        else 'call' if  (x['MA_9_tend_diff'] == -2 and x['tend'] == 0)  else 0, axis =1)

    df['preco'] = df.apply(lambda x: x['Close'], axis =1)

    df['stop'] = df.apply(lambda x: x['High'] + (x['High'] - x['Low'])  if x['acao']=='sell' else x['Low'] - (x['High'] - x['Low']) if x['acao'] == 'call' else 0, axis =1)



    ##Excluir seguidos
    lista = df['acao'].values

    tempo_lista = 10
    for num, i in enumerate(lista):
        #print(num,i)
        if i != 0:
            for i in range(tempo_lista):
                try:
                    n_num = num + i + 1
                    lista[n_num] = 0
                except:
                    pass
    
    df['acao'] = lista

    ultima_linha = df.tail(1)
    preco_entrada = ultima_linha['preco']
    stop = ultima_linha['stop']
    acao = ultima_linha['acao']
    hora_fechamento = hora_fechamento

    if acao == 'call':
        comprar(ativo, stop,hora_fechamento)
    elif acao == 'sell':
        vender(ativo, stop,hora_fechamento)
    else:
        print(ultima_linha)





In [2]:
while True:
    data_e_hora_atuais = datetime.now()
    hora_fechamento = data_e_hora_atuais + timedelta(minutes = 50)
    hora_fechamento = int(hora_fechamento.timestamp())
    hora_teste = str(data_e_hora_atuais.minute)[-1:] + str(data_e_hora_atuais.second)  
    if hora_teste == '959' or hora_teste == '459':
        verificar(ativo, hora_fechamento)
        time.sleep(200)

2022-12-25 08:39:59.001003


KeyboardInterrupt: 

In [3]:
data_e_hora_atuais = datetime.now()

In [8]:
str(data_e_hora_atuais)

'2022-12-25 09:36:32.723920'

In [9]:
str(data_e_hora_atuais + timedelta(minutes = 50))

'2022-12-25 10:26:32.723920'

In [11]:
int(data_e_hora_atuais.timestamp())

1671971792